In [1]:
import numpy as np
from functools import cmp_to_key

In [2]:
class BWT:
    
    def __init__(self):
        pass
    
    def comparator(self, index1, index2):
        seq_len = len(self.sequence)

        while(True):
            char1 = self.sequence[index1]
            char2 = self.sequence[index2]
            if (char2 < char1):
                return 1
            if (char1 < char2):
                return -1
            index1 = (index1 + 1)%seq_len
            index2 = (index2 + 1)%seq_len
        return 0

    def rotation_with_indices(self, sequence):

        """ Given a pattern/sequence, returns a vector of sequences.
            Other sequences are only rotations of the input sequence. """

        seq_len = len(sequence)
        index_vector = [""]*seq_len
        for i in range(len(index_vector)):
            index_vector[i]=i

        rot_vector = [""] * seq_len
        self.sequence=sequence
        self.vector = index_vector
        index_vector.sort(key=cmp_to_key(self.comparator))
        return index_vector
    
    def rotation(self, sequence):
        
        """ Given a pattern/sequence, returns a vector of sequences. 
            Other sequences are only rotations of the input sequence. """
        
        seq_len = len(sequence)
        rot_vector = [""] * seq_len
        seq = sequence * 2
        
        for i in range(seq_len):
            rot_vector[i] = seq[i: seq_len+i]
            
        return rot_vector
    
    def sorting(self, rot_matrix):
        
        """ Given a vector with rotated sequences, returns a vector of sorted sequences 
            which is a Burrows-Wheeler's matrix. Sequences are sorted in a lexicographic order. """
        
        bw_matrix = sorted(rot_matrix)
        return bw_matrix
    
    def first_last_column(self, num_array, seq):
        
        """ Given a numerical array (offsets) and the original sequence, returns the first column. """
        
        first = []
        last = []
        for i, num in enumerate(num_array):
            first.append(seq[num])
            last.append(seq[(num-1)%len(seq)])
        
        first = ''.join(first)
        last = ''.join(last)
        return first, last
        
  
    def first_column(self, Dict):
        
        """ Given a map of symbols with number of appearance, returns the first column as a dictionary. 
            The first column is a map that has a range of rows prefixed by the symbol . """
        
        first = {}
        totc = 0
        temp = sorted(Dict.items())
        
        for c, count in temp:
            
            first[c] = (totc, totc + count)
            totc += count
            
        return first
     
    def last_column(self, bw_matrix):
        bwt = ''
        for s in bw_matrix:
            bwt+=s[-1]
        return bwt
    
    def B_rank(self, sequence):
        
        """ Given the first/last column, returns a rank for each element in that sequence. 
            Also returns a dictionary with a number of appearing for each element. """
        
        Dict = dict()
        ranks = []
        
        for s in sequence:
            
            if s not in Dict:
                
                Dict[s] = 0
            
            ranks.append(Dict[s])
            Dict[s] += 1
      
        return ranks, Dict
        
    
    def LF_mapping(self, last):
        
        """ Given the last column, returns an original sequence. 
            This is a revers transform! """
        
        ranks, dicts = self.B_rank(last)
        first = self.first_column(dicts)
        
        i = 0
        original = '$'
        
        while last[i] != '$':
            
            s = last[i]
            original = s + original
            i = first[s][0] + ranks[i]
    
        return original


In [3]:
def suffix_array(sequence, period=1):
    
    ''' Given a pattern/sequence, returns only suffix-array's offsets. 
        Taking sub-sequences from an incremental position and sorting them in lexicographic order. '''
        
    s = sorted([(sequence[i:], i) for i in range(0, len(sequence))])
    offsets = list(map(lambda x: x[1], s))
    
    if period>1:
        new_sa = []
        for i in range(0, len(offsets)):
            if i % period == 0:
                new_sa.append(offsets[i])
        return new_sa
    else:
        return offsets

def cat_suffix_array(sa, period):
    new_sa = []
    for i in range(0, len(sa)):
        if i % period == 0:
            new_sa.append(sa[i])
    return new_sa


def tally_matrix(last, period = 1):
    
    ''' Given the last column of the BW matrix, returns tally matrix as a dictionary. 
        Keys are symbols and values are columns of a matrix where each column coresponds 
        to a specific symbol. Tally matrix has size (sequence_length, n_symbols). Removed $ sign. '''
    
    counter = {}
    tally = {}
    
    for sym in last:
        if sym != '$':
            if sym not in counter:

                counter[sym] = 0
                tally[sym] = []
        
    
    for i, symb in enumerate(last):     
        if symb != '$':
            
            counter[symb] += 1 
            if (i % period) == 0:
               
                for key in counter.keys():
                    tally[key].append(counter[key])
                    
        else:
            
            if (i % period) == 0:
               
                for key in counter.keys():
                    tally[key].append(counter[key])
            

    return tally, counter
   


In [4]:
def count(BWT, bwt, pattern):
    
    """ Given the Burrow-Wheeler's transform sequence and a pattern, returns a the first and last
        location in the first column. Still, we don't know what are the indeces for alignment! """
    
    L_rank, L_tot = BWT.B_rank(bwt)
    F = BWT.first_column(L_tot)  
    i = len(pattern)-2
    
    lower_limit, upper_limit = F[pattern[-1]]
    while i >= 0 and upper_limit > 1:
        
        search_symbol = pattern[i]
        j = lower_limit
        
        while j < upper_limit:
            
            if bwt[j] == search_symbol:
                
                lower_limit = F[search_symbol][0] + L_rank[j]
                break
                
            j += 1
            
        if j == upper_limit:
            
            lower_limit = upper_limit
            break
            
        upper_limit -= 1
        
        while bwt[upper_limit] != search_symbol:
            
            upper_limit -= 1
            
        upper_limit = F[search_symbol][0] + L_rank[upper_limit] + 1
        i -= 1
    
    if upper_limit == 1:
        print('The pattern does not appear in the sequence!')
    
    return lower_limit, upper_limit-1  
 
    
    
def fast_count(L_rank, L_tot, F, bwt, pattern, tally, period):
    
    """ Given a BW transform, pattern that we search for and tally matrix, returns the first and last
        location in the first column. Still, we don't know what are the indeces for alignment! """
    
    i = len(pattern)-2
    
    lower_limit, upper_limit = F[pattern[-1]]
    lower_limit -= 1
    upper_limit -= 1
    #print(lower_limit, upper_limit)
    
    while i>=0:
        
        search_symbol = pattern[i]
        if lower_limit%period != 0:
            low_rank  = find_rank(bwt, lower_limit, search_symbol, period, tally)
            #print('low ',low_rank)
        else:
            low_rank = tally[search_symbol][lower_limit // period]
            #print('Nasao cp low ', low_rank)
            
        #low_rank = tally[search_symbol][lower_limit]
        
        if upper_limit % period != 0:
            high_rank  = find_rank(bwt, upper_limit, search_symbol, period, tally)
            #print('high' , high_rank)
        else:
            high_rank = tally[search_symbol][upper_limit // period]
            #print('Nasao cp high', high_rank)
            
        
        #high_rank = tally[search_symbol][upper_limit]
        
        diff = high_rank - low_rank # The number of appearance for current symbol
        if diff == 0:
            print('The pattern does not appear in the sequence!')
            break
        
        i -= 1
        
        lower_limit = F[search_symbol][0] + low_rank - 1
        upper_limit = F[search_symbol][0] + low_rank + diff - 1
        
        #print('Novi limiti su ', lower_limit, upper_limit)
        
    return lower_limit + 1, upper_limit  

In [5]:
def locate(L_rank, L_tot, F, bwt, f_range, suffix_array, period):
    
    """ Given a complete suffix_array and a range from a first column, returns a list of position 
        indices where a sequence and a pattern are aligned. """
    
    # Do this only if the suffix array is not optimized
    if period == 1:
        
        low_limit = f_range[0]
        up_limit = f_range[1]
        indices = suffix_array[low_limit:up_limit+1]
        return indices 
    
    len_seq = len(bwt)

    indices = []
    # Suffix array is optimized with a period
    for i in range(f_range[0], f_range[1]+1):
        if i % period == 0:
            #print('Postoji ', i)
            indices.append(suffix_array[i//period])
        else:
            #print('Ne postoji ', i)
            row, count = i, 0
            while row % period != 0:
                count += 1
                s = bwt[row]
                #print(s)
                #print(F[s][0], L_rank[row])
                row = F[s][0] + L_rank[row]
            #print( row//period, row, period)
            #print('Postoji sad row=', row, 'rank=', suffix_array[row//period], ' count=', count)
            indices.append((suffix_array[row//period] + count)%len_seq)   
    return indices        
                    

def find_rank(bwt, row, symbol, period, cp):
    
    n_sym_hidden = 0
    row_start  = row
    
    while (row%period) != 0:
        #print('Row ', row)
        
        if bwt[row] == symbol:
            #print('Nasao', bwt[row])
            n_sym_hidden += 1
      
        row -= 1
 
    cp_pos = cp[symbol][row // period] + n_sym_hidden 
    return cp_pos


In [32]:
bw = BWT()
#seq = "abaaba$"
seq = "aabaababa$"

new_sort = bw.rotation_with_indices(seq)
print('New rot (also an offset array) - ', new_sort, '\n')


f, l = bw.first_last_column(new_sort, seq)
print('First column - ', f, '\n')
print("Last column - ", l, '\n')


rot = bw.rotation(seq)
print('Old rot - ', rot,'\n')


sort = bw.sorting(rot)
print('Old sort - ', sort,'\n')


bwt = bw.last_column(sort)
print('Last - ',bwt, '\n')


f, tot = bw.B_rank(l)
print('B ranks - ', f, '\n')

first = bw.first_column(tot)
print('First - ',first, '\n')

T = bw.LF_mapping(l)
print('Reverse - ',T, '\n')


New rot (also an offset array) -  [9, 8, 0, 3, 6, 1, 4, 7, 2, 5] 

First column -  $aaaaaabbb 

Last column -  ab$bbaaaaa 

Old rot -  ['aabaababa$', 'abaababa$a', 'baababa$aa', 'aababa$aab', 'ababa$aaba', 'baba$aabaa', 'aba$aabaab', 'ba$aabaaba', 'a$aabaabab', '$aabaababa'] 

Old sort -  ['$aabaababa', 'a$aabaabab', 'aabaababa$', 'aababa$aab', 'aba$aabaab', 'abaababa$a', 'ababa$aaba', 'ba$aabaaba', 'baababa$aa', 'baba$aabaa'] 

Last -  ab$bbaaaaa 

B ranks -  [0, 0, 0, 1, 2, 1, 2, 3, 4, 5] 

First -  {'$': (0, 1), 'a': (1, 7), 'b': (7, 10)} 

Reverse -  aabaababa$ 



In [ ]:
pattern = "aba"

n = count(bw, l, pattern)
print('Count normal method: ',n,'\n')

cp, tally = tally_matrix(l, 3)
print('Tally matrix ', cp, '\n')

m = fast_count(bw, l, pattern, cp, 3)
print('Count with tally matrix with(out) optimization: ' ,m, '\n')

offsets = suffix_array(seq, 3)
print('Suffix array ', offsets, '\n')

loc = locate(bw, l, m, offsets, 3)
print('Alignment Indices: ',loc, '\n')


In [6]:
import re
import sys, threading


def read_file(fpath):
    
    """ Givem a .txt file return a list of integers """
    
    file = open(fpath, "r")
    data = []
    
    
    while True:
        line = file.readline()
        if not line: 
            break
        data.append(int(line))
    
    return data


def read_seq(fpath):
    
    sys.setrecursionlimit(10**7) # max depth of recursion
    threading.stack_size(2**27)  # new thread will get stack of such size

    ca_file = open(fpath)
    ca_sequence = ca_file.read()
    ca_sequence = ca_sequence + '$'
    #ca_sequence = ca_sequence.replace("\n",'')
    #ca_sequence = ca_sequence.replace("N",'')

    #print(ca_sequence)
    
    '''
    ca_sequence_new = ""
    for item in ca_sequence.split("\n"):
        count+=1
        if "ref" in item:
            print (item.strip())
        else:
            ca_sequence_new = ca_sequence_new + item

    ca_sequence_new = ca_sequence_new.replace("\n",'')
    '''

    ca_sequence = re.sub(">ref.*\n?", "", ca_sequence)
    ca_sequence = ca_sequence.replace("\n",'')
    ca_sequence = ca_sequence.replace("N",'')
    
    return ca_sequence


In [7]:
coffea_arabica_path = '13443_ref_Cara_1.0_chr1c.fa'
data = read_file("sorted_ca.txt")

In [36]:
print(data[:10])
print(seq[-1])
print(len(seq))
print(max(data))

[50390919, 48061065, 42114805, 48061066, 9356445, 42114806, 48061067, 33955828, 7265287, 9356446]
$
10
50390919


In [8]:
import time
seq = read_seq(coffea_arabica_path)

In [38]:
print(seq[-5:])

CTCT$


In [39]:
bw = BWT()
f, l = bw.first_last_column(data, seq)

In [40]:
print('First column - ', f[:10], '\n')
print("Last column - ", l[:10], '\n')

First column -  $AAAAAAAAA 

Last column -  TCTAGAACGA 



In [ ]:
L_rank, L_tot = bw.B_rank(l)
print('B ranks - ', L_rank[:10], '\n')

In [44]:
F = bw.first_column(L_tot)
print('First - ',F, '\n')

First -  {'$': (0, 1), 'A': (1, 15903823), 'C': (15903823, 25169900), 'G': (25169900, 34468427), 'T': (34468427, 50390920)} 



In [66]:
offsets = cat_suffix_array(data, period = 254)

pattern = "ATGCATG"

cp, tally = tally_matrix(l, 512)

In [46]:
print('Tally matrix ', cp['A'][:10], '\n')

Tally matrix  [0, 335, 704, 1076, 1449, 1838, 2210, 2565, 2924, 3282] 



In [48]:
m = fast_count(L_rank, L_tot, F, l, pattern, cp, 512)
print('Count with tally matrix with(out) optimization: ' ,m, '\n')

Count with tally matrix with(out) optimization:  (13700038, 13706566) 



In [70]:
loc = locate(L_rank, L_tot, F, l, m, offsets, 254)
print('Alignment Indices: ',len(loc), '\n')
#print(suffix_array[row//period], row//period, row, period)


Alignment Indices:  6529 



In [17]:
def find_indices(data, seq, pattern, sa_factor=1, tally_factor=1):
    
    bw = BWT()
    f, l = bw.first_last_column(data, seq)
    L_rank, L_tot = bw.B_rank(l)
    F = bw.first_column(L_tot)
    
    # Suffix array
    offsets = cat_suffix_array(data, period = sa_factor)
    
    # Tally matrix
    cp, tally = tally_matrix(l, tally_factor)
    
    # Start counting time
    start_time = time.clock()
      
    # Range where pattern occurs
    m = fast_count(L_rank, L_tot, F, l, pattern, cp, tally_factor)
    
    # Locations 
    loc = locate(L_rank, L_tot, F, l, m, offsets, sa_factor)
    
    # Print time
    print('Time needed: ', round(time.clock() - start_time, 4), 'seconds!\n')
    

In [ ]:
pattern = "ATGCATG"
find_indices(data, seq, pattern, sa_factor = 254, tally_factor = 8)